<H1 align='center'>Predicting Gender from name using LSTM</H1>
<p align='justify'>This is a dead simple code for predicting Gender from name. Its based on a type of recurrent neural network called <B>LSTM</B> or <B>L</B>ong <B>S</B>hort <B>T</B>erm <B>M</B>emory Network. Feature vector was created using n-gram. Dataset was collected from https://data.world/howarder/gender-by-name . This dataset also has a probability column which shows extreme level of accuracy. So far I got 82% in training and testing. This is a work in progress which may improve gradually later. Hope someone may get some help from it. The method shown here is nowhere near perfect and may even be bad. So I hope people would suggest better things. But its really wonderful to see how this little piece of code that ran may be for 10-15 minutes can be this accurate.</p>
So we begin by importing the necessary modules.

In [ ]:
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D,Embedding,LSTM,Dense, Activation
from keras.callbacks import ModelCheckpoint

Now we load out dataset and make it lowercase immediately to avoid duplication. What this duplication is will be explained later. 

In [2]:
rd=pd.read_csv('name_gender.csv')
rd['name']=rd['name'].str.lower()

<p align='justify'>Now we talk about what N-gram basically is. Say you have a word "Apple". The N of N-gram is actually a number. So a 2 gram representation would be ['Ap','pp','pl','le']. 3-gram would be ['App','ple'] and so on. Now why did I chose this? I actually googled once about this and the concept of n-gram was in one of the search results. It clicked my mind that a name like 'Razia' would have 'ia' at the end which would sound female(I dont know the naming scheme of all countries.) and a name like "george" would have a "ge" which would sound male. Well this logic is not the best thing there is of course but I tried with this approach. The first function returns a list of N-grams.</p>

In [3]:
def ngrams(input, n):
  output = []
  for i in range(len(input)-n+1):
    output.append(input[i:i+n])
  return output
def n_gram_dict(names):
    out_dict={}
    temp_arr=[]
    k=2
    for i in names:
        ng=ngrams(i,2)
        for ngram in ng:
            if ngram not in out_dict:
                out_dict[ngram]=k
                k=k+1
    return out_dict

<p align='justify'>the second function is for our machine learning model specifically. What it does is, it creates a dictionary that contains a number against each n-gram. For example, say we had a name 'Imran'. the 2-gram representation would be ['im','mr','ra','an']. Now machine learning works with numbers. So our dictionary for these n-grams would be {'im':2,'mr':3,'ra':4,'an':5} and so on.k=2 was chosen randomly to make sure our padding doesnt mess around with real data.</p>

In [4]:
#convert each ngram to index
dictionary_of_ngrams=n_gram_dict(rd.name)
list_feature=[]
for name in rd.name:
    ng_name=ngrams(name,2)
    ngram_token=[dictionary_of_ngrams[ng] for ng in ng_name]
    list_feature.append(ngram_token)
feature_vector=pad_sequences(list_feature)
target=pd.get_dummies(rd.gender)
train_X,test_X,train_Y,test_Y=train_test_split(feature_vector,target,test_size=0.2)

<p align='justify'>Now we create our dictionary. Then we convert each name into n-grams then replace each n-gram with its index and that would be our feature vector.We also added padding means the smaller names had less number of n-grams than bigger names so we filled those with 0. We also convert our target vector to one hot representation.</p> 

In [ ]:
model=Sequential()
model.add(Embedding(100, 40,input_length = train_X.shape[1], dropout = 0.2))
model.add(Conv1D(32,(2,),activation='relu'))
model.add(LSTM(100,dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adadelta',metrics = ['accuracy'])
print(model.summary())

<p align='justify'>Our model consists of an Embedding layer and then a 1-D convolution layer which was placed after several experimentation and was seen to have positive effects on accuracy.Then our LSTM layer and later a softmax layer for output. 3 optimizers were tested 'adam','adadelta' and 'rmsprop' where adadelta and adam showed best results during training.</p>

In [6]:
checkpointer = ModelCheckpoint(filepath='weights_name.hdf5', verbose=1, save_best_only=True)
model.fit(train_X,train_Y,batch_size=100,nb_epoch=10,validation_split=0.1,callbacks=[checkpointer])
with open('model_name.json','w') as f:
    f.write(model.to_json())

c:\users\shovon\appdata\local\programs\python\python36\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 68418 samples, validate on 7602 samples
Epoch 1/10
68418/68418 [==============================] - 69s 1ms/step - loss: 0.4841 - acc: 0.7497 - val_loss: 0.4181 - val_acc: 0.7969

Epoch 00001: val_loss improved from inf to 0.41806, saving model to weights_name.hdf5
Epoch 2/10
68418/68418 [==============================] - 61s 892us/step - loss: 0.4258 - acc: 0.7911 - val_loss: 0.4153 - val_acc: 0.7928

Epoch 00002: val_loss improved from 0.41806 to 0.41535, saving model to weights_name.hdf5
Epoch 3/10
68418/68418 [==============================] - 62s 902us/step - loss: 0.4162 - acc: 0.7973 - val_loss: 0.3981 - val_acc: 0.8103

Epoch 00003: val_loss improved from 0.41535 to 0.39812, saving model to weights_name.hdf5
Epoch 4/10
68418/68418 [==============================] - 61s 893us/step - loss: 0.4127 - acc: 0.8011 - val_loss: 0.3978 - val_acc: 0.8135

Epoch 00004: val_loss improved from 0.39812 to 0.39781, saving model to weights_name.hdf5
Epoch 5/10
68418/68418 [=============

In [7]:
# time for some testing
model.evaluate(test_X,test_Y,batch_size=100)

19005/19005 [==============================] - 4s 225us/step


[0.38637625516613544, 0.8168376720883099]

The evaluation shows a score of 0.82 which is not bad for may be an 1 hour experiment.

In [11]:
def predict_gender(name):
    gender=['Female','Male']
    name=name.lower()
    ng_name=ngrams(name,2)
    ngram_token=[dictionary_of_ngrams[ng] for ng in ng_name]
    feature_vector=pad_sequences([ngram_token],maxlen=14)
    pred=model.predict(feature_vector)
    result=np.argmax(pred)
    print("name:",name,"Gender",gender[result])
predict_gender('Abir')
predict_gender('Amrin')
predict_gender('Razia')
predict_gender('Michel')
predict_gender('Michelle')
predict_gender('Bappy')
predict_gender('Rasim')
predict_gender('Muhaimin')
predict_gender('Imran')
predict_gender('Vicky')
predict_gender('Mohona')
predict_gender('Sadia')
predict_gender('Troyee')
predict_gender('Rushafi')
predict_gender('Irtiza')




name: abir Gender Male
name: amrin Gender Male
name: razia Gender Female
name: michel Gender Male
name: michelle Gender Female
name: bappy Gender Male
name: rasim Gender Male
name: muhaimin Gender Male
name: imran Gender Male
name: vicky Gender Male
name: mohona Gender Female
name: sadia Gender Female
name: troyee Gender Female
name: rushafi Gender Male
name: irtiza Gender Female


Here we test some random names. pad_sequences function ahas a maxlen of 14 which was taken from training data padding.